In [176]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_cs
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
#import cPickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn import svm
from sklearn.svm import SVC, LinearSVC # SVM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
#from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.tree import DecisionTreeClassifier 
#from sklearn.learning_curve import learning_curve
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, train_test_split 

In [177]:
def learning_curve(estimator, title, X, y, ylim=None, cv=None,n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("The Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

## Process input files (tweets, labels and features) into a dataset

In [178]:
##################
# join tweets and labels
##################
ground = pd.read_csv("./input/ground_truth.txt", sep='\t').drop_duplicates()
raw = pd.read_csv("./input/raw_tweets.txt", sep='\t',encoding='latin-1').drop_duplicates()
data = ground.merge(raw, on="Tweet ID", how="left").drop(["Tweet ID"], axis=1)
print(ground.shape, raw.shape, data.shape)
data.head(10)

(395, 2) (397, 2) (397, 2)


,Label,Tweet Text
0,Neutral,#AbortionLaws #P2 Q&A on Planned Parenthood co...
1,Against Abortion,MT @Maralm0413 : How can they still be in busi...
2,Against Abortion,@GarnerMeg Why do u consider giving birth to a...
3,Against Abortion,Black student claims she forced to get #aborti...
4,Against Abortion,Baby Angela Born With Anencephaly Defies the...
5,Against Abortion,#WomenBetrayed : The Real Stories Rally & Lobb...
6,Against Abortion,Let us ask that every unborn come to be loved ...
7,Against Abortion,Babies are a gift from God. Please choose life...
8,Against Abortion,how are you handling the babies after an #abor...
9,Against Abortion,Need a ProLife Real Estate Agent? We'll find 1...


In [179]:
#################
#tag the message to be positive or negative
#################
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

In [180]:
hotel_rev = data["Tweet Text"]

data2 = []
sid = SentimentIntensityAnalyzer()
for sentence in hotel_rev:
     print(sentence)
     ss = sid.polarity_scores(sentence)
     for k in ss:
        print('{0}: {1},'.format(k, ss[k]))
     max = 0
     tag = ''
     i = 0
     for k in ss:
        if i == 1:
            max = ss[k]
            tag = 'negative'
        if i == 3 and ss[k]>max:
            tag = 'positive'
        if i == 3 and ss[k] == max:
            tag = ' '
        i += 1
     if tag == 'positive':
        sentence += ' Positive'
     if tag == 'negative':
        sentence += ' Negative'
     data2.append(sentence.lower())#lower case
     #print(max, k, sentence)
     print()
     print()
#print(data2)
data["Tweet Text"] = data2

#AbortionLaws #P2 Q&A on Planned Parenthood controversy in Ohio: Federal law prevents Med... http:// bit.ly/1GUzdO1 #Abortion #Uniteblue
neu: 0.925,
neg: 0.0,
compound: 0.0772,
pos: 0.075,


MT @Maralm0413 : How can they still be in business? #PPSellsBabyParts #ProLife #PlannedButcherhood pic.twitter.com/W9pAotcnLR #DefundPP #PJNET
neu: 1.0,
neg: 0.0,
compound: 0.0,
pos: 0.0,


@GarnerMeg Why do u consider giving birth to a living #preemie who was neglected and died an #abortion ? http://www. in.gov/judiciary/opin ions/pdf/07221601tac.pdf É #PurviPatel
neu: 0.644,
neg: 0.265,
compound: -0.6808,
pos: 0.091,


Black student claims she forced to get #abortion at #PlannedParenthood Ó... http:// fb.me/6wUuSN3M0
neu: 0.8,
neg: 0.2,
compound: -0.4588,
pos: 0.0,


Baby Angela  Born With Anencephaly  Defies the Odds  Turns 1 Year Old http:// buff.ly/1CSqsPd #abortion #prolife pic.twitter.com/15gPjTj3fi
neu: 1.0,
neg: 0.0,
compound: 0.0,
pos: 0.0,


#WomenBetrayed : The Real Stories Rally & Lobb

neu: 0.744,
neg: 0.0,
compound: 0.6712,
pos: 0.256,


Make sure #antichoice lawmakers & the media don't ignore us. Sign the petition 2 say NO 2 abortion bans: http:// nar.al/3et #StopTheBans
neu: 0.685,
neg: 0.126,
compound: 0.1222,
pos: 0.189,


@ABCPolitics Agree w/ @matthewjdowd . Pence did nothing to help Trump. Kaine did his job 3-issues #1 abortion #2 ClintonFDN #3 Trump taxes.
neu: 0.808,
neg: 0.091,
compound: 0.0624,
pos: 0.101,


@johniadarola yea but places like Indiana gives us TONS OF SHIT if you consider abortion .
neu: 0.584,
neg: 0.27,
compound: -0.5789,
pos: 0.146,


Fuck your non- abortion alternatives. #VPDebate #ReproRights
neu: 0.632,
neg: 0.368,
compound: -0.5423,
pos: 0.0,


Just Some of the Reasons Pro- Abortion Hillary Clinton Must Never Become President http://www. lifenews.com/2016/10/14/jus t-some-of-the-reasons-pro-abortion-hillary-clinton-must-never-become-president/#.WAGVw5sp5vk.twitter É
neu: 1.0,
neg: 0.0,
compound: 0.0,
pos: 0.0,


Judge Explains The Da

In [181]:
##################
# extract features
##################
features = []
for i in range(1,5):
    filename = "./input/memos/Memo{0}.txt".format(i)
    with open(filename) as f:
        content = f.readlines()
    features += [line.strip("\n") for line in content]
features[:10]
print(len(features))
print(features)

131
['Horrible Anti-Abortion', "Woman's choice", "Woman's choice #prochoice ", '#humansrights', '#humansrights @ Positive', '#humanrights + well done', '#human rights + #CzarnyProtest', 'well done', '#CzarnyProtest', '#CzarnyProtest @ Positive  ', '#StandWithPP @ Positive', '#StandWithPP', '#shoutyourabortion', '#shoutyourabortion + #StandWithPP', 'trust women', '#balckmonday ', '#standwithpp', '#Prochoice = #Prolife', '#ProChoice is #Prolife ', 'support', '#standwithPP', 'dangerous', '#antichoice', 'domestic terrorism', '#reprorights', 'reprorights @ Positive', '#EndAbortion', '#EndAbortion @ positive', '#UnbornLivesMatter @ positive', '#EndAbortion + #UnbornLivesMatter', 'Join to end abortion', 'women regret abortion', '#unbornlivesmatter @ positive', 'abortion kills @ positive', 'abortion rights', 'abortion rights @ negative', '#billnye @ negative', '#billnye + deplorable', 'planned parenthood selling ', '#PPisevil', 'black genocide', '#PPisevil + black genocide', 'abortion shameful

In [182]:
##################
# calculate feature values
##################

def get_feat(x):
    result = []
    for feat in features:
        if '+' in feat:
            #print(1)
            feat = feat.split(' + ')
            count = 0
            for item in feat:
                if item in x:
                    count += 1 
                else:
                    count += 0
            if count >= 2:
                result += [1] 
            else:
                result += [0] 
        elif '@' in feat:
            #print(2)
            feat = feat.split(' @ ')
            count = 0
            for item in feat:
                if item in x:
                    count += 1 
                else:
                    count += 0
            if count >= 2:
                result += [1] 
            else:
                result += [0]
        elif '=' in feat:
            #print(4)
            feat = feat.split(' = ')
            count = 0
            for item in feat:
                if item in x:
                    count += 1 
                else:
                    count += 0
            if count >= 1:
                result += [1] 
            else:
                result += [0]
        elif 'is' in feat:
            #print(4)
            feat = feat.split(' is ')
            count = 0
            for item in feat:
                if item in x:
                    count += 1 
                else:
                    count += 0
            if count >= 1:
                result += [1] 
            else:
                result += [0]
        else:
            #print(3)
            if feat in x:
                result += [1] 
            else:
                result += [0]
        #print()
    return result

values = list(map(lambda x: get_feat(x), data["Tweet Text"]))

sd = []
for j in range(0,len(values[0])):
    sum = 0
    for i in range(0,len(values)):
        if values[i][j] == 1:
            sum += 1
    if sum <= 5:
        sd.append(j)

        
        
print(len(values[0]))
print(sd)
print(len(sd))

count = 0
for i in range(0,len(sd)):
    j = sd[i]-count
    [r.pop(j) for r in values]
    features.pop(j)
    count += 1
    
print(len(values[0]))
print(len(features))
        
        
print(values)

131
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 21, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 93, 94, 95, 96, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]
116
15
15
[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [183]:
L = [[1,2,3,4],
     [5,6,7,8],
     [9,1,2,3]]
[r.pop(1) for r in L]
print(L)

[[1, 3, 4], [5, 7, 8], [9, 2, 3]]


In [184]:
##################
# merge the label with feature values
##################

# convert the labels to numbers
convert_dict = {"Neutral":1, "Against Abortion":0, "For Abortion":2}
labels = [convert_dict[x] for x in data["Label"]]
# merge
training_data = pd.merge(data, pd.DataFrame(values, columns=features), left_index=True, right_index=True)
training_data = training_data.drop("Tweet Text", axis=1)
training_data["Label"] = labels
print(training_data.shape)
training_data.head(10)

(397, 16)


,Label,#standwithpp,support,#antichoice,#reprorights,#abortion,#prolife,#prochoice,#antichoice,killing,abortion @ negative,poland,babies + killed + abortion,unborn,#unbornlivesmatter,abortion
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
3,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
4,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
8,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1
9,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


## Train, test, evaluate a classifier

In [185]:
#linear model

In [253]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm
X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:, 1:], training_data.iloc[:, 0], test_size=0.1, random_state=0)
clf = lm.LogisticRegression(penalty="l1").fit(X_train, y_train)
clf.score(X_test, y_test)

/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.725

In [255]:
import numpy as np
from sklearn import linear_model

clf = linear_model.SGDClassifier(max_iter=1000)

X = training_data.iloc[:, 1:]
y = training_data.iloc[:, 0]
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
max = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
    if scores[i] > max:
        max = scores[i]
print('Score:', sumscore/len(scores))
print('Max Score:', max)

Total tweets classified: 397
Score: 0.667371794871795
Max Score: 0.9


In [ ]:
#multinomial NB

In [188]:
X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:, 1:], training_data.iloc[:, 0], test_size=0.1, random_state=0)

clf = MultinomialNB()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
score = f1_score(y_test, predictions, average='micro')
print(score)

0.7


In [266]:
X = training_data.iloc[:, 1:]
y = training_data.iloc[:, 0]
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
max = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
    if scores[i] > max:
        max = scores[i]
print('Score:', sumscore/len(scores))
print('Max Score:', max)

Total tweets classified: 397
Score: 0.6573076923076925
Max Score: 0.9


In [190]:
#Bernoulli NB

In [243]:
X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:, 1:], training_data.iloc[:, 0], test_size=0.1, random_state=0)

clf = BernoulliNB(binarize=0.0)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
score = f1_score(y_test, predictions, average='micro')
print(score)

0.7


In [298]:
X = training_data.iloc[:, 1:]
y = training_data.iloc[:, 0]
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
max = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
    if scores[i] > max:
        max = scores[i]
print('Score:', sumscore/len(scores))
print('Max Score:', max)

Total tweets classified: 397
Score: 0.6648076923076924
Max Score: 0.825


In [80]:
# SVM kernel: rbf

In [245]:
X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:, 1:], training_data.iloc[:, 0], test_size=0.1, random_state=0)

clf = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
score = f1_score(y_test, predictions, average='micro')
print(score)

0.65


In [246]:
X = training_data.iloc[:, 1:]
y = training_data.iloc[:, 0]
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
max = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
    if scores[i] > max:
        max = scores[i]
print('Score:', sumscore/len(scores))
print('Max Score:', max)

Total tweets classified: 397
Score: 0.6446794871794872
Max Score: 0.825


In [83]:
#SVM kernel: linear

In [247]:
X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:, 1:], training_data.iloc[:, 0], test_size=0.1, random_state=0)

clf = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
score = f1_score(y_test, predictions, average='micro')
print(score)

0.675


In [248]:
X = training_data.iloc[:, 1:]
y = training_data.iloc[:, 0]
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
max = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
    if scores[i] > max:
        max = scores[i]
print('Score:', sumscore/len(scores))
print('Max Score:', max)

Total tweets classified: 397
Score: 0.6572435897435899
Max Score: 0.825


In [86]:
#kn

In [249]:
X_train, X_test, y_train, y_test = train_test_split(training_data.iloc[:, 1:], training_data.iloc[:, 0], test_size=0.1, random_state=0)

clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
score = f1_score(y_test, predictions, average='micro')
print(score)

0.525


In [250]:
X = training_data.iloc[:, 1:]
y = training_data.iloc[:, 0]
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
max = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
    if scores[i] > max:
        max = scores[i]
print('Score:', sumscore/len(scores))
print('Max Score:', max)

Total tweets classified: 397
Score: 0.5494871794871795
Max Score: 0.7250000000000001


In [89]:
#without memos as a feature

In [232]:
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier',  linear_model.SGDClassifier(max_iter=1000)) ])

X = data.iloc[:]['Tweet Text'].values
y = data.iloc[:]['Label'].values
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
print('Score:', sumscore/len(scores))

Total tweets classified: 397
Score: 0.6519230769230769


In [233]:
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier',  MultinomialNB()) ])

X = data.iloc[:]['Tweet Text'].values
y = data.iloc[:]['Label'].values
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
print('Score:', sumscore/len(scores))

Total tweets classified: 397
Score: 0.5430769230769232


In [234]:
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier',  BernoulliNB(binarize=0.0)) ])

X = data.iloc[:]['Tweet Text'].values
y = data.iloc[:]['Label'].values
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
print('Score:', sumscore/len(scores))

Total tweets classified: 397
Score: 0.532948717948718


In [235]:
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier',  svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)) ])

X = data.iloc[:]['Tweet Text'].values
y = data.iloc[:]['Label'].values
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
print('Score:', sumscore/len(scores))

Total tweets classified: 397
Score: 0.41673076923076924


In [251]:
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier',  svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True,
    tol=0.001, verbose=False)) ])

X = data.iloc[:]['Tweet Text'].values
y = data.iloc[:]['Label'].values
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
print('Score:', sumscore/len(scores))

Total tweets classified: 397
Score: 0.6518589743589744


In [237]:
pipeline = Pipeline([
    ('vectorizer',  CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier',  KNeighborsClassifier(n_neighbors=1)) ])

X = data.iloc[:]['Tweet Text'].values
y = data.iloc[:]['Label'].values
kf = KFold(n_splits=10)
kf.get_n_splits(X)

scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)

    #confusion += confusion_matrix(test_y, predictions)
    score = f1_score(y_test, predictions, average='micro')
    scores.append(score)

print('Total tweets classified:', len(training_data))
sumscore = 0.0
for i in range(0,len(scores)):
    sumscore += scores[i]
print('Score:', sumscore/len(scores))

Total tweets classified: 397
Score: 0.5441025641025641
